In [ ]:
# import dependacies
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pymongo
import requests
import pandas as pd

In [ ]:
# create path and open browser window
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

# establish url
url = 'https://quotes.toscrape.com/'

# visit site
browser.visit(url)

# grab page html
html = browser.html

# create soup object
soup = BeautifulSoup(html,'html.parser')

In [ ]:
# create the initial list to store the data
quotes_list = []

# loop to go thru pages while next button count is greater than zero
while len(browser.links.find_by_partial_text('Next ')) > 0:
    # grab page html
    html = browser.html
    # create soup object
    soup = BeautifulSoup(html,'html.parser')
    # isolate the quote boxes for scraping
    boxes = soup.find_all('div',class_='quote')
    
    # for loop to click "about author" > get data > back > get box data
    for box in boxes:
        # initialize the mini dictionary
        quote_mini = {}
        
        # identify where to click on "about author"
        target = box.a['href']
        # click button
        browser.links.find_by_href(target).click()
        
        # get page html
        html = browser.html
        
        # create a soup object
        soup = BeautifulSoup(html,'html.parser')
        
        # add values for author to the mini dict
        quote_mini['author_name'] = soup.find('h3',class_='author-title').text
        quote_mini['birth_date'] = soup.find('span',class_='author-born-date').text
        quote_mini['birth_place'] = soup.find('span',class_='author-born-location').text
        quote_mini['description'] = soup.find('div',class_='author-description').text.replace('\n', '')
        
        # click back button
        browser.back()
        
        # Grab quote box values for mini dict
        
        # add quote to the mini dict
        quote_mini['quote_text'] = box.span.text
        
        # remove extra spaces, commas, and new line text that is not needed
        quote_tags = box.div.text.replace('\n',',').split(',')
        do_not_want = ['','            Tags:','            ']
        
        # add quote to the mini dict
        quote_mini['quote_tags'] = [tag for tag in quote_tags if tag not in do_not_want]

        # append completed mini dict to the quotes list
        quotes_list.append(quote_mini)
        
    # click next button to move to next page    
    browser.links.find_by_text('Next ').click()

browser.quit()

In [ ]:
# create connection to mongo database
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [ ]:
# create new data base
db = client.quotes_db

In [ ]:
# drop the collection if it already exists
db.quotes.drop()
# instert our list of mini dictionaries from scraping
db.quotes.insert_many(quotes_list)

In [ ]:
# show all quotes in collection
list_of_quotes = list(db.quotes.find())
#print(list_of_quotes)

In [ ]:
# use list of dict to create dataframe
quote_df = pd.DataFrame(list_of_quotes)

# drop mongo id field
quote_df = quote_df.drop(['_id'],axis=1)

# change index value to = quote id
quote_df['quote_id'] = quote_df.index

In [ ]:
# Isolate tags df
tags = quote_df['quote_tags']

# break the tags list in each row into individal rows
tags = quote_df.explode('quote_tags')
tags = tags[['quote_tags','quote_id']]
tags = tags.set_index('quote_id')

# rename column to tags
tags.rename(columns = {'quote_tags':'tags'},inplace=True)